In [ ]:
%load_ext autoreload 
%autoreload 2

In [ ]:
import json
from privacypacking.utils.utils import load_logs
import pandas as pd
from experiments.ray.analysis import load_tasks, load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment, load_scheduling_queue
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve, SyntheticPolynomialCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

from omegaconf import OmegaConf
from pathlib import Path
from collections import defaultdict

In [ ]:
from privacypacking.budget import ALPHAS
from privacypacking.utils.utils import add_workload_args_to_results

Let's see what curves are allocated by different schedulers

In [ ]:
# /home/pierre/privacypacking/logs/ray/run_and_report_2022-04-12_10-19-10


df = load_latest_ray_experiment()
# df = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2022-04-07_09-58-57"))
# df = load_latest_ray_experiment("/home/pierre/privacypacking/logs/ray/run_and_report_2022-04-07_10-07-26")



In [ ]:
df.columns

In [ ]:
df = add_workload_args_to_results(df)

In [ ]:
df.describe()

In [ ]:
df.epsilon_min_avg.unique()

In [ ]:
df.epsilon_min_std.unique()

In [ ]:
df.range_std.unique()

In [ ]:
df.range_avg.unique()

df.describe

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.query("epsilon_min_avg == 0.05 and range_std == 0 and range_avg == 0.05").sort_values(["epsilon_min_std", "total_tasks", "alpha_std"]),
    x="alpha_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    facet_col="epsilon_min_std",
    facet_row="total_tasks",
    title="We keep the avg size (0.05) and change the variance (not only zero)."
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.query("epsilon_min_avg == 0.05 and range_std == 0.03 and range_avg == 0.05").sort_values(["epsilon_min_std", "total_tasks", "alpha_std"]),
    x="alpha_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    facet_col="epsilon_min_std",
    facet_row="total_tasks",
    title="We keep the avg size (0.05) and change the variance (not only zero). Rang std = 0.03 instead of 0"
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.query("epsilon_min_avg == 0.05 and epsilon_min_std == 0.01 and alpha_std == 4").sort_values(["range_std", "total_tasks", "range_avg"]),
    x="range_avg",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    facet_col="range_std",
    facet_row="total_tasks",
    title="We keep the avg size (0.05) and variance (0.01) and alpha std (4). We play with range."
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.query("epsilon_min_avg == 0.05 and epsilon_min_std == 0.01 and total_tasks == 2000").sort_values(["range_std", "alpha_std", "range_avg"]),
    x="range_avg",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    facet_col="range_std",
    facet_row="alpha_std",
    title="We keep the avg size (0.05) and variance (0.01) and total tasks (2k). We play with range."
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.query("epsilon_min_avg == 0.05 and epsilon_min_std == 0 and total_tasks == 2000").sort_values(["range_std", "alpha_std", "range_avg"]),
    x="range_avg",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    facet_col="range_std",
    facet_row="alpha_std",
    title="We keep the avg size (0.05) and variance (0) and total tasks (2k). We play with range."
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.query("alpha_std == 0 and epsilon_min_avg == 0.05 and total_tasks == 2000").sort_values(["range_std", "range_avg", "epsilon_min_std"]),
    x="epsilon_min_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    facet_col="range_std",
    facet_row="range_avg",
    title="We keep a single alpha (alpha std == 0) and total tasks (2k). epsilon_min_avg == 0.05.  We play with the rest."
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.query("alpha_std == 8 and epsilon_min_avg == 0.05 and total_tasks == 2000").sort_values(["range_std", "range_avg", "epsilon_min_std"]),
    x="epsilon_min_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    markers=True,
    facet_col="range_std",
    facet_row="range_avg",
    title="We keep a single alpha (alpha std == 8) and total tasks (2k). epsilon_min_avg == 0.05.  We play with the rest."
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
df = load_latest_ray_experiment()
# /home/pierre/privacypacking/logs/ray/run_and_report_2022-04-12_17-15-38
# Two alphas only: /home/pierre/privacypacking/logs/ray/run_and_report_2022-04-18_13-52-46

df = add_workload_args_to_results(df)

In [ ]:
df.tasks_path[0]

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.sort_values(["scheduler_metric", "total_tasks", "alpha_std"]),
    x="alpha_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=1200,
    height=1200,
    markers=True,
    facet_col="epsilon_min_avg",
    facet_row="total_tasks",
    title="Diversity in best alpha, for different loads. <br><sup> Other knobs are fixed: epsilon_min_avg: [0.05] \
epsilon_min_std: [0.01]\
range_avg: [0.05]\
range_std: [0.03]\
block_avg: 1\
block_std: 0</sup>"
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
df = load_latest_ray_experiment()
df = add_workload_args_to_results(df)

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.sort_values(["scheduler_metric", "total_tasks", "alpha_std"]),
    x="alpha_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=1200,
    height=1200,
    markers=True,
    facet_col="epsilon_min_avg",
    facet_row="total_tasks",
    title="Diversity in best alpha, for different loads. <br><sup> Other knobs are fixed: epsilon_min_avg: [0.05] \
epsilon_min_std: [0.01]\
range_avg: [0.05]\
range_std: [0.03]\
block_avg: 1\
block_std: 0</sup>"
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
df = load_latest_ray_experiment()
df = add_workload_args_to_results(df)
# /home/pierre/privacypacking/logs/ray/run_and_report_2022-04-14_05-05-32

In [ ]:
fig = px.line(
    # rdf.query("data_lifetime == 50 and max_blocks == 400").sort_values("T"),
    df.sort_values(["scheduler_metric", "total_tasks", "epsilon_min_std"]),
    x="epsilon_min_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=800,
    markers=True,
    # facet_col="epsilon_min_std",
    facet_row="total_tasks",
    title="Diversity in epsilon_min, for different loads. <br><sup> Other knobs are fixed: epsilon_min_avg: [0.05] \
best_alpha: [0]\
range_avg: [0.05]\
range_std: [0.03]\
block_avg: 1\
block_std: 0</sup>"
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
df = load_latest_ray_experiment()
df = add_workload_args_to_results(df)
# /home/pierre/privacypacking/logs/ray/run_and_report_2022-04-14_06-24-16

In [ ]:
fig = px.line(
    df.sort_values(["scheduler_metric", "total_tasks", "range_avg"]),
    x="range_avg",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=800,
    markers=True,
    # facet_col="epsilon_min_std",
    facet_row="total_tasks",
    title="Diversity in range_avg, for different loads. <br><sup> Other knobs are fixed: epsilon_min_avg: [0.05] \
best_alpha: [0]\
epsilon_min_std: [0.01] \
range_avg: [0.05]\
range_std: [0.03]\
block_avg: 1\
block_std: 0</sup>"
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
df = load_latest_ray_experiment()
df = add_workload_args_to_results(df)
# /home/pierre/privacypacking/logs/ray/run_and_report_2022-04-14_09-05-31

In [ ]:
fig = px.line(
    df.sort_values(["scheduler_metric", "total_tasks", "range_avg"]),
    x="range_avg",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=800,
    markers=True,
    # facet_col="epsilon_min_std",
    facet_row="total_tasks",
    title="Diversity in range_avg, for different loads. <br><sup> Other knobs are fixed: epsilon_min_avg: [0.05] \
best_alpha: [all]\
epsilon_min_std: [0.01] \
range_avg: [0.05]\
range_std: [0.03]\
block_avg: 1\
block_std: 0</sup>"
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
df = load_latest_ray_experiment()
df = add_workload_args_to_results(df)
# /home/pierre/privacypacking/logs/ray/run_and_report_2022-04-14_09-29-33

In [ ]:
fig = px.line(
    df.sort_values(["scheduler_metric", "total_tasks", "range_std"]),
    x="range_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=800,
    markers=True,
    facet_col="alpha_std",
    facet_row="total_tasks",
    title="Diversity in range_std, for different loads. <br><sup> Other knobs are fixed: epsilon_min_avg: [0.05] \
best_alpha: [0 or all]\
epsilon_min_std: [0.01] \
range_avg: [0.01]\
range_std: [0.03]\
block_avg: 1\
block_std: 0</sup>"
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
df = load_latest_ray_experiment()
df = add_workload_args_to_results(df)

In [ ]:
fig = px.line(
    df.sort_values(["scheduler_metric", "total_tasks", "block_std"]),
    x="block_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=800,
    markers=True,
    # facet_col="alpha_std",
    facet_row="total_tasks",
    title="Diversity in number of blocks requested, for different loads. <br><sup> Other knobs are fixed: epsilon_min_avg: [0.05] \
best_alpha: [0]\
epsilon_min_std: [0.01] \
range_avg: [0.01]\
range_std: [0.03]\
block_avg: 10</sup>"
)
fig.update_yaxes(rangemode="tozero")
fig

In [ ]:
df = load_latest_ray_experiment()
df = add_workload_args_to_results(df)
# Recomputing after every task this time! To be smart about blocks

In [ ]:
fig = px.line(
    df.query("total_tasks >= 0").sort_values(["scheduler_metric", "total_tasks", "block_std"]),
    x="block_std",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=1200,
    markers=True,
    # facet_col="alpha_std",
    facet_row="total_tasks",
    title="Diversity in number of blocks requested, for different loads. <br><sup> Other knobs are fixed: epsilon_min_avg: [0.05] \
best_alpha: [0]\
epsilon_min_std: [0.01] \
range_avg: [0.01]\
range_std: [0.03]\
block_avg: 10</sup>"
)
fig.update_yaxes(rangemode="tozero")
fig